In [56]:

import torch
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

In [66]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model = torch.load('G:/내 드라이브/League-of-Legends-Predicter-main/RNN/best_model.pth',  map_location=device)

cuda:0


In [67]:
testX = [[51325,0,68,13.6,614,140,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,45298,0,62,12.4,609,120,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,85,59,84,145,497,24,64,61,523,412]]
testX = torch.FloatTensor(testX)
testY = [[1]]
testY = torch.FloatTensor(testY)

In [68]:
print(testX.shape)
print(testY.shape)

torch.Size([1, 70])
torch.Size([1, 1])


In [69]:
model

Net(
  (rnn): LSTM(70, 1, batch_first=True)
  (fc): Linear(in_features=1, out_features=1, bias=True)
)

In [71]:
model.eval()
with torch.no_grad():
    testX = testX.to(device)
    prediction = model(testX)
    correct_prediction = torch.argmax(prediction, 1) == testY
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)